# 1. Initialize

In [2]:
from numpy.random import seed
seed(1)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.applications import Xception
from keras.models import Sequential, Model
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras import regularizers
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

np.random.seed(123)

import gc
import psutil
from tqdm import tqdm

process = psutil.Process(os.getpid())
def print_current_ram():
    print(process.memory_info().rss / 1000000000, 'GB')

# 2. Data

## Ground Truth

In [3]:
df_gt = pd.read_csv('../input/isic-2019/ISIC_2019_Training_GroundTruth.csv', index_col='image')
df_gt

## Training Data

In [5]:
# resizing the images and laoding them into array
imx, imy = 128, 128
image_isic = []
for idx in tqdm(df_gt.index):
    input_file_name = '../input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input/' + idx + '.jpg'
    image_isic.append(np.asarray(Image.open(input_file_name).resize((imx,imy))))

x_isic = np.asarray(image_isic)

del image_isic; gc.collect()

In [6]:
# preprocessing the meta data by one hot encoding
df_md = pd.read_csv('../input/isic-2019/ISIC_2019_Training_Metadata.csv', index_col='image')
df_gt1=df_gt.drop(columns=['UNK'])
df_md=df_md.drop(columns=['lesion_id'])
df = pd.merge(df_gt1, df_md, left_index=True, right_index=True)
df['age_approx'] = pd.qcut(df.iloc[:, 8], 5)
df_dc = pd.get_dummies(df, columns=['sex','anatom_site_general','age_approx'])



y_isic = np.asarray(df_gt)
y_isic1 = np.asarray(df_dc)

In [7]:
# test train split for both  woith and without meta data models
x_train, x_validate, y_train, y_validate = train_test_split(x_isic, y_isic, test_size = 0.1, random_state=123)
x_train, x_validate, y_train1, y_validate1 = train_test_split(x_isic, y_isic1, test_size = 0.1, random_state=123)
del x_isic; del y_isic;del y_isic1 ;gc.collect()

In [8]:
x_tr= pd.DataFrame(y_train1, columns = df_dc.columns.values)
x_te= pd.DataFrame(y_validate1, columns = df_dc.columns.values)
x_train_meta=x_tr.iloc[:,8:].to_numpy()
y_train1=x_tr.iloc[:,0:8].to_numpy()
x_test_meta=x_te.iloc[:,8:].to_numpy()
y_test1=x_te.iloc[:,0:8].to_numpy()

# 3. Model 1: No Metadata and No Class weights

In [9]:
input_shape = (imy, imx, 3)
num_classes = 9

# set optimiser
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

epochs = 50
batch_size = 20

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=5, verbose=1, factor=0.5, min_lr=0.00001)
early_stopping_monitor = EarlyStopping(patience=20, monitor='val_accuracy', restore_best_weights=True)

# image for flipping, shifting, rotating and zoom

datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=90,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        shear_range = 10)

In [10]:
training_shape = (imy,imx,3)
base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)

XCeptionmodel = base_model.output
XCeptionmodel = Flatten()(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionmodel = Dense(128, activation='relu')(XCeptionmodel)
XCeptionmodel = Dropout(0.2)(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionoutput = Dense(num_classes, activation = 'softmax')(XCeptionmodel)
XCeptionmodel = Model(inputs=base_model.input, outputs=XCeptionoutput)

model = XCeptionmodel

for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size),
                    epochs = 1, 
                    validation_data = (x_validate,y_validate),
                    verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, 
                    callbacks=[learning_rate_reduction,early_stopping_monitor])

In [ ]:
model.evaluate(x_validate, y_validate, verbose=1)

In [ ]:
y_proba_1 = model.predict([x_validate])
y_classes_pred_1=y_proba_1.argmax(axis=-1)
y_validate_label_1=y_validate.argmax(axis=-1)
tf.math.confusion_matrix(labels=y_validate_label_1, predictions=y_classes_pred_1).numpy()

In [ ]:
# the function returns balanced class accuracy

def multiclassaccuracy(y_true, y_pred):
    a=y_true
    b=y_pred
    z=np.zeros((8,1))
    k=np.zeros((8,1))
    

    for i in range(len(a)):
        z[a[i]]+=1
        if a[i]==b[i]:
             k[a[i]]+=1

    acc=0
    count=0
    for i in range(8):      
      if z[i]!=0:
        count+=1
        acc+=k[i]/z[i] 

    #print(acc/count)
    return acc/count

In [ ]:
multiclassaccuracy(y_validate_label_1, y_classes_pred_1)

## ##  With  no meta data and  class weights

In [ ]:
class_weight_dict={3: 14, 0: 3, 2: 4, 7: 20, 4: 2, 1: 1, 6: 50, 5:5 }

In [ ]:
training_shape = (imy,imx,3)
base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)

XCeptionmodel = base_model.output
XCeptionmodel = Flatten()(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionmodel = Dense(128, activation='relu')(XCeptionmodel)
XCeptionmodel = Dropout(0.2)(XCeptionmodel)

XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionoutput = Dense(num_classes, activation = 'softmax')(XCeptionmodel)
XCeptionmodel = Model(inputs=base_model.input, outputs=XCeptionoutput)

model = XCeptionmodel

for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(datagen.flow(x_train,y_train, batch_size=batch_size),
                    epochs = 1, 
                    validation_data = (x_validate,y_validate),
                    verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, 
                    callbacks=[learning_rate_reduction,early_stopping_monitor],class_weight=class_weight_dict)

In [ ]:
y_proba_2 = model.predict([x_validate])
y_classes_pred_2=y_proba_2.argmax(axis=-1)
y_validate_label_2=y_validate.argmax(axis=-1)
tf.math.confusion_matrix(labels=y_validate_label_2, predictions=y_classes_pred_2).numpy()

In [ ]:
multiclassaccuracy(y_validate_label_2, y_classes_pred_2)

##  With meta data and no class weights

In [23]:
training_shape = (128,128,3)
from keras.layers import concatenate 
import tensorflow as tf


extra = Sequential()
extra.add(Activation('relu', input_shape=(15,)))
extra1=extra.output


base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)
XCeptionmodel = base_model.output
XCeptionmodel = Flatten()(XCeptionmodel)
XCeptionmodel = BatchNormalization()(XCeptionmodel)

XCeptionmodel = concatenate([XCeptionmodel, extra1],axis=-1)

XCeptionmodel = Dense(128, activation='relu')(XCeptionmodel)
XCeptionmodel = Dropout(0.2)(XCeptionmodel)
XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionoutput = Dense(8, activation = 'softmax')(XCeptionmodel)
XCeptionmodel = Model(inputs=[base_model.input,extra.input], outputs=XCeptionoutput)

model = XCeptionmodel

for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
history = model.fit(datagen.flow([x_train,x_train_meta],y_train1, batch_size=batch_size),
                    epochs = 15, 
                    validation_data = ([x_validate,x_test_meta],y_test1),
                    verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, 
                    callbacks=[learning_rate_reduction,early_stopping_monitor],class_weight=class_weight_dict)

In [ ]:
model_metadata.evaluate(x_validate, y_validate, verbose=1)
y_proba_3 = model.predict([x_validate,x_test_meta])
y_classes_pred_3=y_proba_3.argmax(axis=-1)
y_validate_label_3=y_validate.argmax(axis=-1)
tf.math.confusion_matrix(labels=y_validate_label_3, predictions=y_classes_pred_3).numpy()

In [ ]:
multiclassaccuracy(y_validate_label_3, y_classes_pred_3)

##  With meta data and  class weights

In [ ]:
training_shape = (128,128,3)
from keras.layers import concatenate 
import tensorflow as tf


extra = Sequential()
extra.add(Activation('relu', input_shape=(15,)))
extra1=extra.output


base_model = Xception(include_top=False,weights='imagenet',input_shape = training_shape)
XCeptionmodel = base_model.output
XCeptionmodel = Flatten()(XCeptionmodel)
XCeptionmodel = BatchNormalization()(XCeptionmodel)

XCeptionmodel = concatenate([XCeptionmodel, extra1],axis=-1)

XCeptionmodel = Dense(128, activation='relu')(XCeptionmodel)
XCeptionmodel = Dropout(0.2)(XCeptionmodel)
XCeptionmodel = BatchNormalization()(XCeptionmodel)
XCeptionoutput = Dense(8, activation = 'softmax')(XCeptionmodel)
XCeptionmodel = Model(inputs=[base_model.input,extra.input], outputs=XCeptionoutput)

model_metadata = XCeptionmodel

for layer in base_model.layers:
    layer.trainable = True

In [ ]:
model_metadata.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
history = model_metadata.fit(datagen.flow([x_train,x_train_meta],y_train1, batch_size=batch_size),
                    epochs = 15, 
                    validation_data = ([x_validate,x_test_meta],y_test1),
                    verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, 
                    callbacks=[learning_rate_reduction,early_stopping_monitor],class_weight=class_weight_dict)

In [ ]:
model_metadata.evaluate(x_validate, y_validate, verbose=1)
y_proba_4 = model.predict([x_validate,x_test_meta])
y_classes_pred_4=y_proba_4.argmax(axis=-1)
y_validate_label_4=y_validate.argmax(axis=-1)
tf.math.confusion_matrix(labels=y_validate_label_4, predictions=y_classes_pred_4).numpy()

In [ ]:
multiclassaccuracy(y_validate_label_4, y_classes_pred_4)

## Ensemble predictions from two models

In [ ]:
prob=y_proba_1+y_proba_3
y_classes_pred=prob.argmax(axis=-1)
y_validate_label=y_validate.argmax(axis=-1)
tf.math.confusion_matrix(labels=y_validate_label, predictions=y_classes_pred).numpy()

In [ ]:
multiclassaccuracy(y_validate_label, y_classes_pred)